In [88]:
import git
import os

repo = git.Repo('.', search_parent_directories=True)


os.chdir(repo.working_tree_dir)

In [89]:
import pandas as pd
import numpy as np

In [90]:
predictors = pd.read_csv('data_ingest/data/original/predictors.csv')

In [91]:
predictors.drop(columns=['date', 'sunrise'], inplace=True)

In [92]:
import pycmap
api = pycmap.API(token='<8f086ef3-74b5-44a8-9e11-6135f4cacaf1>')
predictors['time'] = pd.to_datetime(predictors['time'])
predictors['time'] = predictors['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [93]:
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [94]:
alk = pd.DataFrame()
for i in range(1,13):
    alk_month = api.query(
        f'''
        SELECT month, lat, lon, depth, ALK_darwin_clim FROM tblDarwin_Nutrient_Climatology
        WHERE
        lat >= -80 AND lat <= 80 AND
        lon >= -180 AND lon <= 180 AND
        depth > 4 AND depth < 6 AND
        month = {i}
        ORDER BY lat, lon
        '''
    )
    alk = pd.concat([alk, alk_month])

In [95]:
sss = pd.DataFrame() 
for i in months:
    sss_month = api.query(
        f'''
        SELECT [time], lat, lon, sss_smap FROM tblSSS_NRT_cl1
        WHERE
        lat >= -80 AND lat <= 80 AND
        lon >= -180 AND lon <= 180 AND
        [time] >= '2023-{i}-09' AND [time] < '2023-{i}-12'
        ORDER BY [time], lat, lon
        '''
    )
    sss_month['month'] = i
    sss = pd.concat([sss, sss_month])

In [96]:
sst = pd.DataFrame()
for i in months:
    sst_month = api.query(
        f'''
        SELECT [time], lat, lon, sst FROM tblSST_AVHRR_OI_NRT
        WHERE
        lat >= -80 AND lat <= 80 AND
        lon >= -180 AND lon <= 180 AND 
        [time] >= '2023-{i}-10' AND [time] < '2023-{i}-11'
        ORDER BY [time], lat, lon
        '''
    )
    sst_month['month'] = i
    sst = pd.concat([sst, sst_month])

In [97]:
pisces = pd.DataFrame()
for i in months:
    pisces_month = api.query(
        f'''
        SELECT [time], lat, lon, fe, o2, no3, po4, si FROM tblPisces_Forecast_cl1
        WHERE
        lat >= -80 AND lat <= 80 AND
        lon >= -180 AND lon <= 180 AND
        ROUND(lat) = lat AND
        ROUND(lon) = lon AND
        [time] >= '2023-{i}-10' AND [time] < '2023-{i}-11' AND
        depth > 4 AND depth < 6
        ORDER BY [time], lat, lon
        '''
    )
    pisces_month['month'] = i
    pisces = pd.concat([pisces, pisces_month])

In [98]:
sss.head(3)

,time,lat,lon,sss_smap,month
0,2023-01-09T00:00:00,-79.875,-179.875,NaN,01
1,2023-01-09T00:00:00,-79.875,-179.625,NaN,01
2,2023-01-09T00:00:00,-79.875,-179.375,NaN,01


In [99]:
sss.isna().sum()

time               0
lat                0
lon                0
sss_smap    14278769
month              0
dtype: int64

In [100]:
sst.head(3)

,time,lat,lon,sst,month
0,2023-01-10T00:00:00,-79.875,-179.875,NaN,01
1,2023-01-10T00:00:00,-79.875,-179.625,NaN,01
2,2023-01-10T00:00:00,-79.875,-179.375,NaN,01


In [101]:
alk.head(3)

,month,lat,lon,depth,ALK_darwin_clim
0,1,-79.75,-179.75,5,2335.9120
1,1,-79.75,-179.25,5,2335.8413
2,1,-79.75,-178.75,5,2335.7673


In [111]:
pisces.head(3)

,time,lat,lon,fe,o2,no3,po4,si,month
0,2023-01-10T12:00:00,-76,-180,0.000171,373.240845,23.282454,1.687487,55.517113,1
1,2023-01-10T12:00:00,-76,-179,0.000190,374.358765,23.062960,1.666234,55.184734,1
2,2023-01-10T12:00:00,-76,-178,0.000182,374.747345,23.098080,1.672519,55.377731,1


In [103]:
dfs = [alk, sss, sst, pisces]

In [107]:
for df in dfs:
    df.dropna(inplace=True)

In [110]:
for df in dfs:
    df.reset_index(drop=True, inplace=True)

In [105]:
for df in dfs:
    df['month'] = df['month'].astype(int)

In [108]:
for df in dfs:
    print(df.shape)

(1940100, 5)
(18898831, 5)
(7642020, 5)
(472704, 9)


In [112]:
alk1 = alk.copy()
sss1 = sss.copy()
sst1 = sst.copy()
pisces1 = pisces.copy()

In [115]:
def lat_rounder(df):
    if 'time' in df.columns:
        df.drop(columns=['time'], inplace=True)
    df = df.reset_index()
    df['lat_rounded'] = df['lat'].round()
    df['lon_rounded'] = df['lon'].round()

    
    df1 = df.groupby(['month', 'lat_rounded', 'lon_rounded']).mean().reset_index()

    
    return df1

In [116]:
alk2 = lat_rounder(alk1)
sss2 = lat_rounder(sss1)
sst2 = lat_rounder(sst1)
pisces2 = lat_rounder(pisces1)

In [130]:
alk2.head(3)

,month,lat,lon,depth,ALK_darwin_clim
0,1,-80.0,-180.0,5.0,2335.91200
1,1,-80.0,-179.0,5.0,2335.80430
2,1,-80.0,-178.0,5.0,2335.64715


In [118]:
new_dfs = [alk2, sss2, sst2, pisces2]

In [124]:
for df in new_dfs:
    df.drop(columns=['lat', 'lon'], inplace=True)
    df.drop(columns=['index'], inplace=True)
    df.rename(columns={'lat_rounded': 'lat', 'lon_rounded': 'lon'}, inplace=True)
    print(df.shape)


(495792, 6)
(417350, 5)
(502908, 5)
(472704, 9)


In [131]:
def merge_dfs(predictors, dfs):
    i = 0
    for df in dfs:
        predictors = pd.merge(predictors, df, on=['month','lat', 'lon'], how='left')
        i += 1
        print('merged', {i})
    return predictors

In [132]:
print(len(new_dfs))
print(len(new_dfs[1:]))

4
3


In [133]:
yearly_cmap = merge_dfs(alk2, new_dfs[1:])

merged {1}
merged {2}
merged {3}


In [137]:
yearly_cmap.dropna(inplace=True)

In [138]:
len(yearly_cmap)

411179

In [144]:
yearly_cmap.head(3)

,month,lat,lon,alk,sss,sst,fe,o2,no3,po4,si,hours_since_sunrise
185,1,-76.0,-180.0,2335.931050,34.366368,-0.316256,0.000171,373.240845,23.282454,1.687487,55.517113,4
186,1,-76.0,-179.0,2335.771725,34.474505,-0.314383,0.000190,374.358765,23.062960,1.666234,55.184734,4
187,1,-76.0,-178.0,2335.594925,34.318974,-0.386252,0.000182,374.747345,23.098080,1.672519,55.377731,4


In [141]:
yearly_cmap.rename(columns={'ALK_darwin_clim': 'alk', 'sss_smap': 'sss'}, inplace=True)

In [142]:
yearly_cmap.drop(columns=['depth'], inplace=True)

In [143]:
yearly_cmap['hours_since_sunrise'] = 4

In [146]:

import pyarrow as pa
import pyarrow.parquet as pq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.1/27.1 MB 8.7 MB/s eta 0:00:0000:0100:01


In [147]:
yearly = pa.Table.from_pandas(yearly_cmap)
pq.write_table(yearly, 'data_ingest/data/original/yearly_cmap.parquet')